# Getting Weather API Data 
- From open meteo 
- The lat and long are set to Torbay
- Chosen variables are apparent temperature (Deg C) , rain (mm), wind speed (knots), gust (knots) and direction (degrees)
- Put into database

In [11]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["apparent_temperature", "rain", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "is_day"],
	"wind_speed_unit": "kn",
	"forecast_days": 14
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_apparent_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_is_day = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["rain"] = hourly_rain
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Identifing activities 
- classify the values to activities
- make output tables 

In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tabulate import tabulate  # For clean table display

# Ensure 'date' column is in datetime format
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])

# Function to classify wind direction from degrees
def degrees_to_direction(degrees):
    if np.isnan(degrees):
        return None
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    index = int((degrees + 22.5) % 360 // 45)
    return directions[index]

# Apply wind direction classification once
hourly_dataframe['wind_direction_cardinal'] = hourly_dataframe['wind_direction_10m'].apply(degrees_to_direction)

# Function to classify activities
def identify_activity(row):
    if row['is_day'] == 1:
        if row['apparent_temperature'] > 5 and row['wind_speed_10m'] > 1 and row['wind_gusts_10m'] < 50:
            return "Windsurfing"
        if row['wind_gusts_10m'] < 10 and row['apparent_temperature'] > 5:
            return "Surfing"
        if row['wind_gusts_10m'] < 5 and row['rain'] < 1:
            return "Walking"
    return "Reading"

# Apply activity classification once
hourly_dataframe['Activity'] = hourly_dataframe.apply(identify_activity, axis=1)

# Extract the day, day of week, and determine weekends
hourly_dataframe['Date'] = hourly_dataframe['date'].dt.date
hourly_dataframe['Day_of_Week'] = hourly_dataframe['date'].dt.strftime('%A')  # Full day name
hourly_dataframe['is_weekend'] = hourly_dataframe['date'].dt.weekday >= 5

# Daily summary for all activities
daily_summary = hourly_dataframe[hourly_dataframe['is_day'] == 1].groupby('Date').agg(
    Day_of_Week=('Day_of_Week', 'first'),
    Activity=('Activity', lambda x: next((activity for activity in x if activity != "Reading"), "Reading")),
    Mean_Temperature=('apparent_temperature', lambda x: round(x.mean(), 1)),
    Mean_Wind_Speed=('wind_speed_10m', lambda x: round(x.mean(), 1))
).reset_index()

# Highlight weekends
def bold_weekend_rows(row):
    if hourly_dataframe[hourly_dataframe['Date'] == row['Date']]['is_weekend'].iloc[0]:
        return {col: f"\033[1m{row[col]} \033[0m" for col in daily_summary.columns}
    return row

daily_summary = daily_summary.apply(lambda row: pd.Series(bold_weekend_rows(row)), axis=1)

# Display daily activity summary
print("Daily Activity Summary:")
print(tabulate(daily_summary, headers="keys", tablefmt="grid", showindex=False))




Daily Activity Summary:
+-------------+---------------+-------------+--------------------+-------------------+
| Date        | Day_of_Week   | Activity    |   Mean_Temperature |   Mean_Wind_Speed |
+=============+===============+=============+====================+===================+
| 2024-12-17  | Tuesday       | Windsurfing |                5.3 |              10.5 |
+-------------+---------------+-------------+--------------------+-------------------+
| 2024-12-18  | Wednesday     | Reading     |                3.6 |               7.5 |
+-------------+---------------+-------------+--------------------+-------------------+
| 2024-12-19  | Thursday      | Windsurfing |                7.6 |              10.9 |
+-------------+---------------+-------------+--------------------+-------------------+
| 2024-12-20  | Friday        | Reading     |               -0.2 |              11.9 |
+-------------+---------------+-------------+--------------------+-------------------+
| 2024-12-21  | Sat

# Activity specifics.
- The weather for the activities identified. 

In [15]:
# Filter data for the next 14 days
today = datetime.now().date()
next_14_days = hourly_dataframe[(hourly_dataframe['Date'] >= today) &
                                (hourly_dataframe['Date'] <= today + timedelta(days=14))]

# Generalized function to summarize activity days
def summarize_activity(df, activity, metrics):
    activity_df = df[df['Activity'] == activity]
    if activity_df.empty:
        print(f"\nNo {activity} days in the next 14 days.")
    else:
        summary = activity_df.groupby('Date').agg(**metrics).reset_index()
        summary.insert(1, "Day_of_Week", activity_df.groupby('Date')['Day_of_Week'].first().values)
        print(f"\n{activity} Days in the Next 14 Days:")
        print(tabulate(summary, headers="keys", tablefmt="grid", showindex=False))

# Metrics for each activity
windsurfing_metrics = {
    'Mean_Wind_Speed': ('wind_speed_10m', lambda x: round(x.mean(), 1)),
    'Mean_Wind_Gusts': ('wind_gusts_10m', lambda x: round(x.mean(), 1)),
    'Dominant_Wind_Direction': ('wind_direction_cardinal', lambda x: x.value_counts().idxmax())
}

surfing_metrics = {
    'Mean_Temperature': ('apparent_temperature', lambda x: round(x.mean(), 1)),
    'Mean_Wind_Speed': ('wind_speed_10m', lambda x: round(x.mean(), 1))
}

walking_metrics = {
    'Mean_Temperature': ('apparent_temperature', lambda x: round(x.mean(), 1))
}

reading_metrics = {
    'Mean_Temperature': ('apparent_temperature', lambda x: round(x.mean(), 1)),
    'Mean_Rainfall': ('rain', lambda x: round(x.mean(), 1))
}

# Summarize and display results
summarize_activity(next_14_days, "Windsurfing", windsurfing_metrics)
summarize_activity(next_14_days, "Surfing", surfing_metrics)
summarize_activity(next_14_days, "Walking", walking_metrics)
summarize_activity(next_14_days, "Reading", reading_metrics)



Windsurfing Days in the Next 14 Days:
+------------+---------------+-------------------+-------------------+---------------------------+
| Date       | Day_of_Week   |   Mean_Wind_Speed |   Mean_Wind_Gusts | Dominant_Wind_Direction   |
+============+===============+===================+===================+===========================+
| 2024-12-17 | Tuesday       |              10.5 |              22.9 | W                         |
+------------+---------------+-------------------+-------------------+---------------------------+
| 2024-12-19 | Thursday      |              10.9 |              23.3 | SW                        |
+------------+---------------+-------------------+-------------------+---------------------------+
| 2024-12-26 | Thursday      |               7.2 |              18   | NW                        |
+------------+---------------+-------------------+-------------------+---------------------------+

No Surfing days in the next 14 days.

Walking Days in the Next 14 Day